# LightGBM with Optuna tuning
* doc: 
https://github.com/optuna/optuna

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [ ]:
train.head()

In [ ]:
train.columns.to_list()

In [ ]:
train.info()

In [ ]:
train['Survived'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
df=train
for c in df.columns:
    if df[c].dtype=='object': 
        lbl = LabelEncoder()
        df[c]=df[c].fillna('N')
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)
train=df

In [ ]:
df=test
for c in df.columns:
    if df[c].dtype=='object': 
        lbl = LabelEncoder()
        df[c]=df[c].fillna('N')
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)
test=df

In [ ]:
target = train['Survived']
data = train.drop(['Survived','PassengerId','Name','Ticket'],axis=1)

In [ ]:
columns=data.columns.to_list()
print(columns)

In [ ]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param =   {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'objective': trial.suggest_categorical('objective',['regression','rmse','binary']),  
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 1.0),
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        "bagging_freq": trial.suggest_int('bagging_freq', 1, 10),
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        "verbosity": -1,
    }
    model = lgb.LGBMClassifier(**param)      
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=32)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# shows the evolution of the search
optuna.visualization.plot_slice(study)

In [ ]:
# parameter interactions on an interactive chart.
optuna.visualization.plot_contour(study, params=['num_leaves','objective','learning_rate'])

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial=study.best_trial.params
print(Best_trial)

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv")
print(sample.shape)

In [ ]:
test[columns]

In [ ]:
preds = np.zeros((sample.shape[0]))
kf = KFold(n_splits=5,random_state=48,shuffle=True)
for trn_idx, test_idx in kf.split(train[columns],target):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=target.iloc[trn_idx],target.iloc[test_idx]
    model = lgb.LGBMClassifier(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits   ###### predict_proba
    rmse=mean_squared_error(y_val, model.predict(X_val),squared=False)
    print(rmse)

In [ ]:
print(preds.shape)
print(preds[0])

In [ ]:
subm = sample
subm['Survived'] = preds.astype(int)
subm.to_csv('submission.csv',index=False)
subm

In [ ]:
subm['Survived'].value_counts()